In [12]:
import math
import random
import numpy as np
%run TSP.ipynb

In [13]:
drilling_problem_data = TSP('a280.tsp')
print(drilling_problem_data)

NOME : a280
PROBLEMA : drilling problem (Ludwig)
TIPO : TSP
TAMNHO: 280
CALCULO : EUC_2D


In [14]:
def calcular_fitness(individuo, tsp):
    fitness = 0
    for i in range(len(individuo)-1):
        p1 = tsp.data[str(individuo[i])]
        p2 = tsp.data[str(individuo[i+1])]
        
        fitness += calcular_distancia(p1, p2)
    return fitness

In [15]:
def calcular_distancia(p1, p2):
    xd = p1[0] - p1[1];
    yd = p2[0] - p2[1];
    dij = int(math.sqrt( xd*xd + yd*yd) );
    return dij

In [16]:
def gerar_solucao(tamanho_dna):
    dna = list(range(1, (tamanho_dna + 1)))
    random.shuffle(dna)
    dna.append(dna[0])
    return dna

In [17]:
def gerar_populacao_aleatoria(tamanho_populacao, tamanho_dna):
    populacao = []
    for indice in range(1, (tamanho_populacao+1)):
        populacao.append({"indice": str(indice), "dna": gerar_solucao(tamanho_dna)})
    return populacao

In [18]:
def get_individuo_por_indice(indice, populacao):
    return list(filter(lambda ind: ind['indice'] == str(indice), populacao))[0]

In [19]:
def crossover(pai, mae):
    # Valores de corte para o DNA
    inicio = random.randint(0, len(pai['dna'])-1)
    fim = random.randint(inicio+1, len(pai['dna']))
    
    # Cria genes iniciais para filho a partir do pai
    dna_filho = pai['dna'][inicio:fim]
    
    # Contabiliza quantos genes faltam
    genes_faltando = len(pai['dna']) - len(dna_filho) - 1
    
    genes_adicionados = 0
    i = 0
    
    # Atribuí genes restantes a partir da mãe
    while genes_adicionados < genes_faltando:
        gene = mae['dna'][i]
        
        if gene not in dna_filho:
            dna_filho.append(gene)
            genes_adicionados += 1
    
        i += 1
    
    # Aponta para ponto inicial
    dna_filho.append(dna_filho[0])
        
    return dna_filho

In [20]:
def mutar(dna, valor_mutacao):
    dna_antigo = dna[:-1]
    dna_novo = np.zeros(shape=len(dna_antigo), dtype=int)
    
    indices_para_mutacao = []
    genes_a_ser_mutado = random.sample(dna_antigo, valor_mutacao)
   
    for gene in genes_a_ser_mutado:
        mutacao = random.randint(1, len(dna_antigo))
        
        while mutacao in dna_novo:
            mutacao = random.randint(1, len(dna_antigo))
        
        dna_novo[dna_antigo.index(gene)] = mutacao
          
    # Contabiliza quantos genes faltam
    genes_faltando = len(dna_antigo) - valor_mutacao
    
    genes_adicionados = 0
    
    i = 0
    # Atribuí genes restantes a partir da mãe
    while genes_adicionados < genes_faltando:
        if dna_novo[i] == 0:
            for gene in dna_antigo:
                if gene not in dna_novo:
                    dna_novo[i] = gene
                    genes_adicionados += 1
                    break
        i += 1    
    
    dna_novo = dna_novo.tolist()
    
    # Aponta para ponto inicial
    dna_novo.append(dna_novo[0])
    return dna_novo

In [21]:
def algoritmo_genetico(limite_de_geracoes=5000, limite_sem_melhora=500, 
                       tamanho_populacao=3, taxa_cruzamento=0.5, taxa_mutacao=0.02 , 
                       tsp=TSP('a280.tsp')):
    # Cria uma população aleatóriamente, conforme `tamanho_populacao`.
    # Cada individuo representa uma possível solução
    populacao = gerar_populacao_aleatoria(tamanho_populacao, tsp.tamanho)

    tamanho_populacao_cruzamento = math.ceil(tamanho_populacao * taxa_cruzamento)
    
    valor_mutacao = math.ceil(tsp.tamanho * taxa_mutacao)
    
    indice_total = tamanho_populacao + 1
    
    geracao_atual = 0
    geracoes_sem_melhora = 0
    
    melhor_individuo = None
    
    # geracao_atual < limite_de_geracoes and
    while geracoes_sem_melhora < limite_sem_melhora:
        # Armazena fitness da populução atual
        populacao_fitness = {}

        # Calcula fitness da populução atual
        for individuo in populacao:
            populacao_fitness[individuo['indice']] = calcular_fitness(individuo['dna'], tsp)

        nova_populacao = []
        melhor_individuo_da_geracao = None
        
        # Pega % da população com melhor fitness e cria descendentes a partir deles
        for rank, indice in enumerate(sorted(populacao_fitness, key=populacao_fitness.get)[:tamanho_populacao_cruzamento]):
            individuo = get_individuo_por_indice(indice, populacao)
            
            if rank == 0:
                melhor_individuo_da_geracao = individuo

            # Adiciona individuopara próxima população
            nova_populacao.append(individuo)
            
            # Seleciona mãe aleatoriamente dentro da população
            pai = individuo
            mae = random.choice(populacao)
            
            # Cria filho
            filho = {}
            filho['indice'] = str(indice_total)
            filho['dna'] = mutar(crossover(pai, mae), valor_mutacao)
            indice_total += 1
            
            # Adiciona filho a nova população
            nova_populacao.append(filho)

        # Substitui população antiga com nova polação de descendentes
        while len(nova_populacao) < len(populacao):
            nova_populacao.append(random.choice(populacao))
        
        populacao = nova_populacao
        
        
        # Ajusta valores para próxima geração
        if melhor_individuo != melhor_individuo_da_geracao:
            melhor_individuo = melhor_individuo_da_geracao
            geracoes_sem_melhora = 0
        else:
            geracoes_sem_melhora += 1
        
        print("Geração: {}\tLimite sem melhora: {}\t Fitness: {}".format(geracao_atual, geracoes_sem_melhora, calcular_fitness(melhor_individuo['dna'], tsp)))
        
        geracao_atual += 1
        
        

In [22]:
algoritmo_genetico()

Geração: 0	Limite sem melhora: 0	 Fitness: 38042
Geração: 1	Limite sem melhora: 1	 Fitness: 38042
Geração: 2	Limite sem melhora: 0	 Fitness: 38024
Geração: 3	Limite sem melhora: 1	 Fitness: 38024
Geração: 4	Limite sem melhora: 0	 Fitness: 38002
Geração: 5	Limite sem melhora: 0	 Fitness: 37987
Geração: 6	Limite sem melhora: 0	 Fitness: 37975
Geração: 7	Limite sem melhora: 1	 Fitness: 37975
Geração: 8	Limite sem melhora: 0	 Fitness: 37970
Geração: 9	Limite sem melhora: 1	 Fitness: 37970
Geração: 10	Limite sem melhora: 0	 Fitness: 37925
Geração: 11	Limite sem melhora: 1	 Fitness: 37925
Geração: 12	Limite sem melhora: 0	 Fitness: 37877
Geração: 13	Limite sem melhora: 0	 Fitness: 37825
Geração: 14	Limite sem melhora: 1	 Fitness: 37825
Geração: 15	Limite sem melhora: 0	 Fitness: 37773
Geração: 16	Limite sem melhora: 0	 Fitness: 37700
Geração: 17	Limite sem melhora: 1	 Fitness: 37700
Geração: 18	Limite sem melhora: 2	 Fitness: 37700
Geração: 19	Limite sem melhora: 3	 Fitness: 37700
Geração: 2

Geração: 163	Limite sem melhora: 7	 Fitness: 36876
Geração: 164	Limite sem melhora: 8	 Fitness: 36876
Geração: 165	Limite sem melhora: 9	 Fitness: 36876
Geração: 166	Limite sem melhora: 0	 Fitness: 36856
Geração: 167	Limite sem melhora: 1	 Fitness: 36856
Geração: 168	Limite sem melhora: 2	 Fitness: 36856
Geração: 169	Limite sem melhora: 3	 Fitness: 36856
Geração: 170	Limite sem melhora: 4	 Fitness: 36856
Geração: 171	Limite sem melhora: 5	 Fitness: 36856
Geração: 172	Limite sem melhora: 6	 Fitness: 36856
Geração: 173	Limite sem melhora: 7	 Fitness: 36856
Geração: 174	Limite sem melhora: 8	 Fitness: 36856
Geração: 175	Limite sem melhora: 0	 Fitness: 36830
Geração: 176	Limite sem melhora: 1	 Fitness: 36830
Geração: 177	Limite sem melhora: 2	 Fitness: 36830
Geração: 178	Limite sem melhora: 0	 Fitness: 36776
Geração: 179	Limite sem melhora: 1	 Fitness: 36776
Geração: 180	Limite sem melhora: 2	 Fitness: 36776
Geração: 181	Limite sem melhora: 3	 Fitness: 36776
Geração: 182	Limite sem melhora

Geração: 323	Limite sem melhora: 16	 Fitness: 36396
Geração: 324	Limite sem melhora: 0	 Fitness: 36394
Geração: 325	Limite sem melhora: 1	 Fitness: 36394
Geração: 326	Limite sem melhora: 2	 Fitness: 36394
Geração: 327	Limite sem melhora: 3	 Fitness: 36394
Geração: 328	Limite sem melhora: 4	 Fitness: 36394
Geração: 329	Limite sem melhora: 5	 Fitness: 36394
Geração: 330	Limite sem melhora: 6	 Fitness: 36394
Geração: 331	Limite sem melhora: 7	 Fitness: 36394
Geração: 332	Limite sem melhora: 0	 Fitness: 36386
Geração: 333	Limite sem melhora: 0	 Fitness: 36344
Geração: 334	Limite sem melhora: 1	 Fitness: 36344
Geração: 335	Limite sem melhora: 2	 Fitness: 36344
Geração: 336	Limite sem melhora: 3	 Fitness: 36344
Geração: 337	Limite sem melhora: 0	 Fitness: 36340
Geração: 338	Limite sem melhora: 0	 Fitness: 36332
Geração: 339	Limite sem melhora: 1	 Fitness: 36332
Geração: 340	Limite sem melhora: 2	 Fitness: 36332
Geração: 341	Limite sem melhora: 3	 Fitness: 36332
Geração: 342	Limite sem melhor

Geração: 482	Limite sem melhora: 61	 Fitness: 36242
Geração: 483	Limite sem melhora: 62	 Fitness: 36242
Geração: 484	Limite sem melhora: 63	 Fitness: 36242
Geração: 485	Limite sem melhora: 64	 Fitness: 36242
Geração: 486	Limite sem melhora: 65	 Fitness: 36242
Geração: 487	Limite sem melhora: 66	 Fitness: 36242
Geração: 488	Limite sem melhora: 67	 Fitness: 36242
Geração: 489	Limite sem melhora: 68	 Fitness: 36242
Geração: 490	Limite sem melhora: 69	 Fitness: 36242
Geração: 491	Limite sem melhora: 70	 Fitness: 36242
Geração: 492	Limite sem melhora: 71	 Fitness: 36242
Geração: 493	Limite sem melhora: 72	 Fitness: 36242
Geração: 494	Limite sem melhora: 73	 Fitness: 36242
Geração: 495	Limite sem melhora: 74	 Fitness: 36242
Geração: 496	Limite sem melhora: 75	 Fitness: 36242
Geração: 497	Limite sem melhora: 76	 Fitness: 36242
Geração: 498	Limite sem melhora: 77	 Fitness: 36242
Geração: 499	Limite sem melhora: 78	 Fitness: 36242
Geração: 500	Limite sem melhora: 79	 Fitness: 36242
Geração: 501

Geração: 641	Limite sem melhora: 50	 Fitness: 36073
Geração: 642	Limite sem melhora: 51	 Fitness: 36073
Geração: 643	Limite sem melhora: 52	 Fitness: 36073
Geração: 644	Limite sem melhora: 53	 Fitness: 36073
Geração: 645	Limite sem melhora: 54	 Fitness: 36073
Geração: 646	Limite sem melhora: 55	 Fitness: 36073
Geração: 647	Limite sem melhora: 56	 Fitness: 36073
Geração: 648	Limite sem melhora: 57	 Fitness: 36073
Geração: 649	Limite sem melhora: 58	 Fitness: 36073
Geração: 650	Limite sem melhora: 59	 Fitness: 36073
Geração: 651	Limite sem melhora: 60	 Fitness: 36073
Geração: 652	Limite sem melhora: 61	 Fitness: 36073
Geração: 653	Limite sem melhora: 62	 Fitness: 36073
Geração: 654	Limite sem melhora: 63	 Fitness: 36073
Geração: 655	Limite sem melhora: 64	 Fitness: 36073
Geração: 656	Limite sem melhora: 65	 Fitness: 36073
Geração: 657	Limite sem melhora: 66	 Fitness: 36073
Geração: 658	Limite sem melhora: 67	 Fitness: 36073
Geração: 659	Limite sem melhora: 68	 Fitness: 36073
Geração: 660

...

Geração: 2195	Limite sem melhora: 271	 Fitness: 35717
Geração: 2196	Limite sem melhora: 272	 Fitness: 35717
Geração: 2197	Limite sem melhora: 273	 Fitness: 35717
Geração: 2198	Limite sem melhora: 274	 Fitness: 35717
Geração: 2199	Limite sem melhora: 275	 Fitness: 35717
Geração: 2200	Limite sem melhora: 276	 Fitness: 35717
Geração: 2201	Limite sem melhora: 277	 Fitness: 35717
Geração: 2202	Limite sem melhora: 278	 Fitness: 35717
Geração: 2203	Limite sem melhora: 279	 Fitness: 35717
Geração: 2204	Limite sem melhora: 280	 Fitness: 35717
Geração: 2205	Limite sem melhora: 281	 Fitness: 35717
Geração: 2206	Limite sem melhora: 282	 Fitness: 35717
Geração: 2207	Limite sem melhora: 283	 Fitness: 35717
Geração: 2208	Limite sem melhora: 284	 Fitness: 35717
Geração: 2209	Limite sem melhora: 285	 Fitness: 35717
Geração: 2210	Limite sem melhora: 286	 Fitness: 35717
Geração: 2211	Limite sem melhora: 287	 Fitness: 35717
Geração: 2212	Limite sem melhora: 288	 Fitness: 35717
Geração: 2213	Limite sem mel

Geração: 2349	Limite sem melhora: 65	 Fitness: 35690
Geração: 2350	Limite sem melhora: 66	 Fitness: 35690
Geração: 2351	Limite sem melhora: 67	 Fitness: 35690
Geração: 2352	Limite sem melhora: 68	 Fitness: 35690
Geração: 2353	Limite sem melhora: 69	 Fitness: 35690
Geração: 2354	Limite sem melhora: 70	 Fitness: 35690
Geração: 2355	Limite sem melhora: 71	 Fitness: 35690
Geração: 2356	Limite sem melhora: 72	 Fitness: 35690
Geração: 2357	Limite sem melhora: 73	 Fitness: 35690
Geração: 2358	Limite sem melhora: 74	 Fitness: 35690
Geração: 2359	Limite sem melhora: 75	 Fitness: 35690
Geração: 2360	Limite sem melhora: 76	 Fitness: 35690
Geração: 2361	Limite sem melhora: 77	 Fitness: 35690
Geração: 2362	Limite sem melhora: 78	 Fitness: 35690
Geração: 2363	Limite sem melhora: 79	 Fitness: 35690
Geração: 2364	Limite sem melhora: 80	 Fitness: 35690
Geração: 2365	Limite sem melhora: 81	 Fitness: 35690
Geração: 2366	Limite sem melhora: 82	 Fitness: 35690
Geração: 2367	Limite sem melhora: 83	 Fitness:

Geração: 2504	Limite sem melhora: 105	 Fitness: 35632
Geração: 2505	Limite sem melhora: 106	 Fitness: 35632
Geração: 2506	Limite sem melhora: 107	 Fitness: 35632
Geração: 2507	Limite sem melhora: 108	 Fitness: 35632
Geração: 2508	Limite sem melhora: 109	 Fitness: 35632
Geração: 2509	Limite sem melhora: 110	 Fitness: 35632
Geração: 2510	Limite sem melhora: 111	 Fitness: 35632
Geração: 2511	Limite sem melhora: 112	 Fitness: 35632
Geração: 2512	Limite sem melhora: 113	 Fitness: 35632
Geração: 2513	Limite sem melhora: 114	 Fitness: 35632
Geração: 2514	Limite sem melhora: 115	 Fitness: 35632
Geração: 2515	Limite sem melhora: 116	 Fitness: 35632
Geração: 2516	Limite sem melhora: 117	 Fitness: 35632
Geração: 2517	Limite sem melhora: 118	 Fitness: 35632
Geração: 2518	Limite sem melhora: 119	 Fitness: 35632
Geração: 2519	Limite sem melhora: 120	 Fitness: 35632
Geração: 2520	Limite sem melhora: 121	 Fitness: 35632
Geração: 2521	Limite sem melhora: 122	 Fitness: 35632
Geração: 2522	Limite sem mel

Geração: 2656	Limite sem melhora: 257	 Fitness: 35632
Geração: 2657	Limite sem melhora: 258	 Fitness: 35632
Geração: 2658	Limite sem melhora: 259	 Fitness: 35632
Geração: 2659	Limite sem melhora: 260	 Fitness: 35632
Geração: 2660	Limite sem melhora: 261	 Fitness: 35632
Geração: 2661	Limite sem melhora: 262	 Fitness: 35632
Geração: 2662	Limite sem melhora: 263	 Fitness: 35632
Geração: 2663	Limite sem melhora: 264	 Fitness: 35632
Geração: 2664	Limite sem melhora: 265	 Fitness: 35632
Geração: 2665	Limite sem melhora: 266	 Fitness: 35632
Geração: 2666	Limite sem melhora: 267	 Fitness: 35632
Geração: 2667	Limite sem melhora: 268	 Fitness: 35632
Geração: 2668	Limite sem melhora: 269	 Fitness: 35632
Geração: 2669	Limite sem melhora: 270	 Fitness: 35632
Geração: 2670	Limite sem melhora: 271	 Fitness: 35632
Geração: 2671	Limite sem melhora: 272	 Fitness: 35632
Geração: 2672	Limite sem melhora: 273	 Fitness: 35632
Geração: 2673	Limite sem melhora: 274	 Fitness: 35632
Geração: 2674	Limite sem mel

IndexError: index 281 is out of bounds for axis 0 with size 281